In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import featurewiz as gwiz

In [ ]:
data = pd.read_csv('dataset.csv')
data.head()


In [ ]:
data.SMILES.count()

In [ ]:
data.Solubility.count()

In [ ]:
#MessingAround
#Chem.MolFromSmiles(data.SMILES[0])
#Chem.MolFromSmiles('ClCC(Cl)(Cl)Cl')
#m = Chem.MolFromSmiles('ClCC(Cl)(Cl)Cl')
#m.GetNumAtoms()

Calculating molecular descriptors in rdkit

In [ ]:
mol_list= []
for element in data.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

In [ ]:
len(mol_list)

In [ ]:
mol_list[:5]

https://github.com/dataprofessor/code/blob/master/python/cheminformatics_predicting_solubility.ipynb
Calculate molecular descriptors
To predict LogS (log of the aqueous solubility), the study by Delaney makes use of 4 molecular descriptors:
cLogP (Octanol-water partition coefficient)
MW (Molecular weight)
RB (Number of rotatable bonds)
AP (Aromatic proportion = number of aromatic atoms / total number of heavy atoms)
Unfortunately, rdkit readily computes the first 3. As for the AP descriptor, we will calculate this by manually computing the ratio of the number of aromatic atoms to the total number of heavy atoms which rdkit can compute.

LogP, MW and RB

In [ ]:
def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df = generate(data.SMILES)
df

Aromatic Proportion

In [ ]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')     

In [ ]:
aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
aromatic_atoms

In [ ]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [ ]:
AromaticAtoms(m)

In [ ]:
desc_AromaticAtoms = [AromaticAtoms(element) for element in mol_list]
desc_AromaticAtoms

No. of heavy atoms

In [ ]:
#single molecule
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
Descriptors.HeavyAtomCount(m)

In [ ]:
#all dataset
desc_HeavyAtomCount = [Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_HeavyAtomCount

Aromatic Proportion Descriptor 

In [ ]:
m = Chem.MolFromSmiles('COc1cccc2cc(C(=O)NCCCCN3CCN(c4cccc5nccnc54)CC3)oc21')
AromaticAtoms(m)/Descriptors.HeavyAtomCount(m)

In [1]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_AromaticProportion

NameError: name 'mol_list' is not defined

In [ ]:
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])
df_desc_AromaticProportion

X matrix (Combining all computed descriptors into 1 dataframe)

In [ ]:
X = pd.concat([df,df_desc_AromaticProportion], axis=1)
X

In [ ]:
data.head()

Y-Matrix

In [ ]:
Y = data.iloc[:,1]
Y

DataSplit

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
Y_train.head()

LINREG

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df.head()

# Instantiate the OneHotEncoder object
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')

# Apply one-hot encoding to the categorical column
encoded = encoder.fit_transform(data[['Name']])

# Convert the encoded data back to a data frame
encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names())

# Print the encoded data frame
print(encoded_df)

HELP!

In [ ]:
model = linear_model.LinearRegression()
#model.fit(X_train, Y_train)